# Quantum machine learning for quantum chemistry problems
In this file we will be taking you through the procces of predicting the atomization energy of a molecule using quantum machine learning. To do this we will first introduce you to the data than we will do some classical machine learning to get a benchmark, and lastly we will be using quantum machine learning.

Setup of the file:
1. Data analysis
2. Classical machine learning
3. Quantum encodings
4. Quantum machine

We assume some basic understanding of quantum computing and python programming to read this file.

In [ ]:
# Pennylane imports
import pennylane as qml

# Torch imports
import torch

# Sklearn imports
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA, KernelPCA
from sklearn.datasets import make_moons
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics.pairwise import laplacian_kernel, rbf_kernel, chi2_kernel

# Numpy imports
import numpy as np

# Mathplotlib imports
from matplotlib import pyplot as plt

# Other imports
from time import time
from scipy.io import loadmat
import random
from matplotlib import cm

# Own imports
from data import numberOfAtoms, numberOfAtomX
from qnn import QNN

np.random.seed(123)
torch.manual_seed(123)

### Support functions

In [ ]:
# Devide the atomization energy into equal buckets
def devideInBuckets(y, classes):
    # The size of a bucket is the total range devded by the amount of classes
    stepSize = ((np.amin(y)-np.amax(y))/classes)*-1
    print("step is:", stepSize)
    
    start = np.amax(y)
    buckets = np.zeros((y.shape[0], 1))
    
    for i in range(len(y)):
        for j in range(classes):
            if y[i][0] <= start-(stepSize*j) and y[i][0] >= start-stepSize*(j+1):
                    buckets[i][0] = j
            else:
                continue
    return buckets

# Return the sum of the coulomb matrix
def coulombSum(x):
    cm_sum = np.zeros((x.shape[0], 1))
    
    for i in range(len(x)):
        cm_sum[i][0] = np.sum(x[i])
        
    return cm_sum

## Dataset
We use the qm7b molecular dataset which is a dataset of 7211 molecules in coulomb matrix representation, with 14 molecular properties per coulomb matrix (molecule). A coulomb matrix in way of representing a molecule computationally and its constructed as follows:

$$\begin{split}
    C_{ij} &= \begin{cases}
              0.5 Z_{i}^{2.4} & \text{if } i = j \\
              & \\
              \dfrac{Z_{i}Z_{j}}{\vert R_{i}-R{j}\vert} & \text{if } i \neq j
              \end{cases},
  \end{split}$$
  
where $i$ and $j$ are the rows and columns of the Coulomb matrix, $Z$ is the nuclear charge of the atoms ($i$ or $j$), and $R$ is the Cartesian coordinates of the atoms ($i$ or $j$). 

In [ ]:
# Load the qm7b dataset
mat = loadmat(file_name="qm7b.mat")

# Remove the information in the data set we don't need
for key in list(mat.keys()):
    if "__" in key:
        mat.pop(key, None)
        
# Print the keys
print(mat.keys())

In [ ]:
# X is the coulomb matrix (7211, 23, 23) Y are the properties (7211, 14)
X = mat['X']
Y = mat['T']

# Resize them for sklearn (7211, 529) (7211, 14)
x = X.reshape(X.shape[0], -1)
y = Y.reshape(Y.shape[0], -1)

## Plotting
Now that we have the data loaded in we can try and make plot with it to see whats going on. We make a couple of different plots of the atomization energy to get a good picture of the data.

In [ ]:
# Some data to plot later
atomization_energy = np.zeros((y.shape[0], 1))


# Isolate the atomization energy which is property 0 of the 14 properties.
for i in range(y.shape[0]):
    atomization_energy[i][0] = y[i][0]

When trying to plot the atomization energy against the coulomb matrix we run into a problem, the coulomb matrix has 529 dimentions. To solve this we can use dimentionallity reducting (PCA).

In [ ]:
pca = PCA(n_components=1)
x1 = pca.fit_transform(x)
print("Explained variance is: ", np.sum(pca.explained_variance_ratio_))

plt.scatter(x1, atomization_energy, color='blue')
plt.show()


Unfortionally when trying to reduce the dimention we get a bad explained variance, meaning that our reduced data is not representative of the original data. Because PCA doens't work well in this case, we can plot things like the number of atoms or the sum of the coulomb matrix against the atomization energy to see if there is any relation.

In [ ]:
# Declare array's for different properties.
coulomb_sum = np.zeros((x.shape[0], 1))
num_atoms = np.zeros((x.shape[0], 1))
num_H_atoms = np.zeros((x.shape[0], 1))
num_C_atoms = np.zeros((x.shape[0], 1))

# Take the sum of he coulomb matrix
for i in range(x.shape[0]):
    coulomb_sum[i][0] = np.sum(x[i])
    
# Path variabels in the functions are a problem for later.
# These functions find the number of atoms and the number of a specific atom in the molecules.
num_atoms = numberOfAtoms(x)
num_H_atoms = numberOfAtomX('H')
num_C_atoms = numberOfAtomX('C')

In [ ]:
# Some plots
fig, axs = plt.subplots(2, 2, )
axs[0][0].scatter(coulomb_sum, atomization_energy, color='blue')
axs[0][1].scatter(num_atoms, atomization_energy, color='red')
axs[1][0].scatter(num_H_atoms, atomization_energy, color='green')
axs[1][1].scatter(num_C_atoms, atomization_energy, color='purple')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(14,10))
scatter = axs.scatter(num_atoms, atomization_energy, color='red')
plt.show()

We can see a nice correlation between the number of atoms and the atomization energy in these plots But there is still some overlap if you would try and use this as input.

In [ ]:
# Combine some of the properties into one array
a = np.concatenate((coulomb_sum, num_atoms), axis=1)
b = np.concatenate((a, num_C_atoms), axis=1)
c = np.concatenate((b, num_H_atoms), axis=1)

print(c.shape)

# PCA to reduce the dimentions
pca = PCA(n_components=2)
c = pca.fit_transform(c)

# Print the shape and the explained variance of the data
print(c.shape)
print(np.sum(pca.explained_variance_ratio_))

# Plot the results of the pcs
fig, axs = plt.subplots(1, 1, )
scatter = axs.scatter(c[:,0], c[:,1], c=atomization_energy[:,0], cmap=cm.jet_r)
colorbar = fig.colorbar(scatter, ax=axs, label = "Atomization energy")
plt.show()

In [ ]:
from math import ceil

fig, axs = plt.subplots(1, 1, figsize=(14,10))
scatter = axs.scatter(num_H_atoms, num_C_atoms, c=atomization_energy, s=60, edgecolors='black', cmap=cm.jet_r)
colorbar = fig.colorbar(scatter, ax=axs, label = "Atomization energy")

plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14,10))
axs = fig.add_subplot(111, projection='3d')
scatter = axs.scatter(num_atoms, num_H_atoms, num_C_atoms, c=y[:,0], s=60, edgecolors='black', cmap=cm.jet_r)
colorbar = fig.colorbar(scatter, ax=axs, label = "(kcal/mol)")
plt.show()

As you can see from the plots the number of atoms and the number of specific atoms are highly correlated with the atomization energy of the molecule. Altough there is still some overlap.

## buckets
We try to turn the regression problem into a classification problem by defining "buckets" (ranges) of the atomizations energy, and predicting those. We define 5 buckets [2, 3, 6, 8, 10] as that our quantum neural network (QNN) is build to handle those classification tasks.

In [ ]:
# The amount of classes
n_classes=3

# Function to devide the atomization in buckets based on the number of classes you want.
buckets = devideInBuckets(atomization_energy, n_classes)

print(buckets[0:10])

# We plot the ranges
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)

# Plot the split
scatter = ax.scatter(num_atoms, atomization_energy, c=buckets, s=60, edgecolors='black', cmap=cm.jet_r)
colorbar = fig.colorbar(scatter, ax=ax, label = "buckets")

# Plot the lines to split the data
for u in range(1, n_classes):
    plt.hlines(((np.amin(atomization_energy)-np.amax(atomization_energy))/n_classes)*u+np.amax(atomization_energy), 4, 23, color='red')

plt.show()

## Classical machine learning
For the cassical machine learning part we use sklearn's MLPclassifier to predict the ranges of atomization energy. We start by just using the coulomb matrix to predict the different buckets we established in one part up.

In [ ]:
# Reshape the buckets from a collumb vector to a row vector
buckets = buckets.reshape(-1)

# Split the data into train test splits
trainx, testx, trainy, testy = train_test_split(x, buckets, test_size=0.25)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

# Train the classifier
CLAS = MLPClassifier().fit(trainx, trainy)
print("----- Class score is:", CLAS.score(testx, testy), "Mean absolute error is:", 
      mean_absolute_error(testy, CLAS.predict(testx)), "-----")

This is already a good result but we can do better.

### Input data
We can try to use different input data to see wether the score improves. For examples from the plots part we saw that the number of atoms in a molecule is strongly correlated with its atomization energy. So we can try different input data and see how they compare against eachother.

In [ ]:
# Plain coulomb matrix
CM = x

# Sum of the coulomb matrix
CMsum = coulombSum(x)

# Number of atoms
NumberOfAtoms = numberOfAtoms(x)

# Number of H atoms
NumberOfHAtoms = numberOfAtomX(atom='H')

# Number of C atoms
NumberOfCAtoms = numberOfAtomX(atom='C')

# Sum of coulomb matrix with the number of atoms
CM_NumAtoms = np.concatenate((CMsum, NumberOfAtoms), axis=1)

# Sum of coulomb matrix with number of atoms and number of H atoms
CM_NumAtoms_H = np.concatenate((CM_NumAtoms, NumberOfHAtoms), axis=1)

# Sum of coulomb matrix with number of atoms and number of H and C atoms
CM_NumAtoms_H_C = np.concatenate((CM_NumAtoms_H, NumberOfCAtoms), axis=1)

# PCA on Sum of coulomb matrix with number of atoms and number of H and C atoms
# Using pca might helps to perserve the important data
P = PCA(n_components=3)
PCAall = P.fit_transform(CM_NumAtoms_H_C)
print(P.explained_variance_ratio_, np.sum(P.explained_variance_ratio_))

inputs = [CM, CMsum, NumberOfAtoms, NumberOfHAtoms, NumberOfCAtoms,  
          CM_NumAtoms, CM_NumAtoms_H, CM_NumAtoms_H_C, PCAall]

names = ["CM", "CMsum", "NumberOfAtoms", "NumberOfHAtoms", "NumberOfCAtoms", "CM_NumAtoms", 
         "CM_NumAtoms_H", "CM_NumAtoms_H_C", "PCAall"]

In [ ]:
# Arrays to store the results of the different input data's
scores = []
mea = []

# Compare each input
for inp in range(len(inputs)):
    # Make training and testing data
    trainx, testx, trainy, testy = train_test_split(inputs[inp], buckets, test_size=0.25)
    print(names[inp], trainx.shape, testx.shape, trainy.shape, testy.shape)
    
    # Train the classifier
    CLAS = MLPClassifier().fit(trainx, trainy)
    
    # Print the results
    print("----- Class score is:", CLAS.score(testx, testy), "Mean absolute error is:", 
          mean_absolute_error(testy, CLAS.predict(testx)), "-----")
    
    # Add the results to the array's
    scores.append(CLAS.score(testx, testy))
    mea.append(mean_absolute_error(testy, CLAS.predict(testx)))

In [ ]:
# Scores plot
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111)
ax.plot(names, scores, color='blue')
ax.scatter(names, scores, color='red')
plt.xticks(rotation=40)

plt.show()

In [ ]:
# Mae plot
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111)
ax.plot(names, mea, color='blue')
ax.scatter(names, mea, color='red')
plt.xticks(rotation=40)

plt.show()

From these results we can quickly see a few things. Firstly we get the best results with data that involves the number of atoms (noa), as we saw in the plots the noa is very correlated with the atomization energy so this is not suprising. Secondly we see that adding extra information besides just noa can increase the score as seen for the last input. I think this is because adding extra information can negate part of the overlap that you get with only the noa.

### Grid search
Now that we have compared the input data against eachother we can now use a grid search to find good hyperparameters. The gridsearch is just a dictionary of different hyperparameters that are tested agains eachother.

In [ ]:
# Parameter grid of different parameter that will be tested against eachother
param_grid = {'hidden_layer_sizes': [(5, ), (10, ), (15, ), (20, ), (25, )], 
              'activation': ['relu', 'logistic', 'tanh', 'identity'], 
              'solver': ['adam', 'sgd'], 
              'learning_rate': ['constant', 'invscaling', 'adaptive'],
              'learning_rate_init': [0.0001, 0.001, 0.01, 0.1]
             }
# Train test datasplit
trainx, testx, trainy, testy = train_test_split(PCAall, buckets, test_size=0.25)

# Classifier
Class = MLPClassifier()

# The grid search and its best parameters and best score
gridsearch = GridSearchCV(CLAS, param_grid, n_jobs=-1).fit(trainx, trainy)
print(gridsearch.best_params_)
print(gridsearch.best_score_)

# Quantum encoding
There are different methods of encoding classical information into a quantum circuit for this notebook we wil be showing angle, dense angle and amplitude encoding. Our qnn uses dense angle encoding but has the option to use aplitude encoding.

### Angle encoding
Angle encoding uses rotations to enode classical information into a quantum circuit. This means N data points can be stored in N qubits, however a advantage of this method is a constant encoding circuit depth as you only need 1 gate per qubit.

$$\vert x\rangle = \bigotimes_{i=1}^n \cos(x_i)|0\rangle + \sin(x_i)|1\rangle .$$


### Dense angle encoding
Dense angle encoding works the same as angle encoding but it it also adds a phase. because of this data can be 2N data can be stored in N qubits.

$$\vert x\rangle = \bigotimes_{i=1}^{\lceil N/2\rceil} \cos(\pi x_{2i-1})|0\rangle + e^{2\pi ix_{2i}}\sin(\pi x_{2i-1})|1\rangle .$$


### Amplitude encoding
The classical information is encoded in the state vector of the system. This allows you to store N data in log_2(N) qubits, but has a bigger circuit depth compared to the previous methods.

$$\vert x\rangle = \frac{1}{||\textbf{x}||}\sum_{i=1}^{2^n} x_i |i\rangle .$$

## Quantum machine learning
Now finally we can go and use quantum machine learning. lucky we already have a quantum neural network class (QNN) made. A quantum neural network consists of 4 parts: 
1. Encoding
2. Layer(s) of rotation gates
3. Measurement
4. Classical optimization

![](img/variational.png)

The encoding in this case is done using dense angle encoding as discussed above. We use zyz rotation gates combined with CNOT's for the layers. We use pennylane to measure the expectation value and classicly optimize the weights (rotations of the gates) using pytorch.

In [ ]:
# This hyperparameter dict is used for the torch optimizer (default: Adam).
optdict = {'lr': 0.001}


qd = qml.device('default.qubit', shots=1000, wires=PCAall.shape[1])

# The qnn splits the data into train and test data.
qnn = QNN(x=PCAall, y=buckets, n_layers=1, n_classes=n_classes, reuploading=True, epochs=20, 
          preperation='angle', qpuRun=False, token=None, batch_size=32, seed=123, quantum_device=qd, **optdict)

# True for tracking the accuracy of the network each epoch
qnn.train(True)
qnn.plotCost()
#print(qnn.weights)